In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import random

In [ ]:
!pwd

In [ ]:
# IS_FISHEYE = False
# CHECKERBOARD = (10,7)
# square_size = 35#mm
# images_glob = '/catkin_ws/src/dataset/calib/fish1280x960A3/03/*.png'
# img_size = (1280, 960)

# IS_FISHEYE = False
# CHECKERBOARD = (10,7)
# square_size = 35#mm
# images_glob = '/catkin_ws/src/dataset/calib/fish640x480A3/01/*.png'
# img_size = (1280, 960)

# CHECKERBOARD = (9,7)
# square_size = 20#mm
# images_glob = '/catkin_ws/src/dataset/calib/fish1280x960/**/*.jpeg'
# img_size = (1280, 960)


# IS_FISHEYE = False 
# CHECKERBOARD = (10,7)
# square_size = 35#mm
# images_glob = '/catkin_ws/src/dataset/calib/picam1296x972A3/01/*.png'
# img_size = (1296,972)


IS_FISHEYE = False 
CHECKERBOARD = (10,7)
square_size = 35#mm
images_glob = '../calib_images/leopard_640x480_01/*.jpg'
img_size = (640,480)

objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= square_size

subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)

calibration_flags = 0
if IS_FISHEYE:
    # calibration_flags |= cv2.fisheye.CALIB_USE_INTRINSIC_GUESS
    calibration_flags |= cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC
    # calibration_flags |= cv2.fisheye.CALIB_CHECK_COND
    calibration_flags |= cv2.fisheye.CALIB_FIX_SKEW
    # calibration_flags |= cv2.fisheye.CALIB_FIX_K1
    # calibration_flags |= cv2.fisheye.CALIB_FIX_K2
    # calibration_flags |= cv2.fisheye.CALIB_FIX_K3
    # calibration_flags |= cv2.fisheye.CALIB_FIX_K4
    # calibration_flags |= cv2.fisheye.CALIB_FIX_INTRINSIC
    # calibration_flags |= cv2.fisheye.CALIB_FIX_PRINCIPAL_POINT
    # calibration_flags |= cv2.fisheye.CALIB_ZERO_DISPARITY
else:
    # calibration_flags |= cv2.CALIB_USE_INTRINSIC_GUESS
    # calibration_flags |= cv2.CALIB_FIX_PRINCIPAL_POINT
    # calibration_flags |= cv2.CALIB_FIX_ASPECT_RATIO
    # calibration_flags |= cv2.CALIB_ZERO_TANGENT_DIST
    # calibration_flags |= cv2.CALIB_FIX_K1
    # calibration_flags |= cv2.CALIB_RATIONAL_MODEL
    # calibration_flags |= cv2.CALIB_THIN_PRISM_MODEL
    # calibration_flags |= cv2.CALIB_FIX_S1_S2_S3_S4
    # calibration_flags |= cv2.CALIB_TILTED_MODEL
    # calibration_flags |= cv2.CALIB_FIX_TAUX_TAUY
    pass


find_corners_flags = 0
find_corners_flags |= cv2.CALIB_CB_ADAPTIVE_THRESH
find_corners_flags |= cv2.CALIB_CB_FAST_CHECK
find_corners_flags |= cv2.CALIB_CB_NORMALIZE_IMAGE




image_paths = glob.glob(images_glob)
print('found {} images'.format(len(image_paths)))

objpoints = [] # 3d point in real world spacec
imgpoints = [] # 2d points in image plane.
_img_shape = None
idx2image_path = {}

for img_path in image_paths:
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (CHECKERBOARD[0],CHECKERBOARD[1]), find_corners_flags)

    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
        idx2image_path[len(objpoints)] = img_path


N_OK = len(objpoints)
print("Found " + str(N_OK) + " valid images for calibration")
if IS_FISHEYE:
    mtx = np.zeros((3, 3))
    dist = np.zeros((4, 1))
    rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
    tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
    ret, mtx, dist, rvecs, tvecs = \
        cv2.fisheye.calibrate(
            objpoints,
            imgpoints,
            img_size,
            mtx,
            dist,
            rvecs,
            tvecs,
            flags = calibration_flags,
            criteria=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
        )
else:
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints,
        imgpoints,
        img_size,
        None,
        None,
        flags = calibration_flags,
    )


print("RET", ret)

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )



print("DIM=" + str(_img_shape[::-1]))
print(mtx, '\n', dist)
print("Camera.fx:", mtx[0][0])
print("Camera.fy:", mtx[1][1])
print("Camera.cx:", mtx[0][2])
print("Camera.cy:", mtx[1][2])
if IS_FISHEYE:
    print('Camera.k1:', dist[0][0])
    print('Camera.k2:', dist[1][0])
    print('Camera.k3:', dist[2][0])
    print('Camera.k4:', dist[3][0])
else:
    print('Camera.k1:', dist[0][0])
    print('Camera.k2:', dist[0][1])
    print('Camera.p1:', dist[0][2])
    print('Camera.p2:', dist[0][3])
    print('Camera.k3:', dist[0][4]) 



In [ ]:
x = []
y = []
u = []
v = []
errors = []
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    for j, p in enumerate(imgpoints[i]):
        diff = p - imgpoints2[j]
        err = np.linalg.norm(diff[0])
        x.append(p[0,0])
        y.append(p[0,1])
        u.append(diff[0,0])
        v.append(diff[0,1])
        errors.append(err)


fig, ((ax,ax2)) = plt.subplots(1,2,figsize=(15,8))

ax.scatter(x, y, c=errors, alpha=0.5, cmap="viridis_r")
ax.axis("scaled")
ax.set_xlim(0, img_size[0])
ax.set_ylim(0, img_size[1])

pos = ax2.quiver(x, y, u, v, errors, cmap="viridis_r")
ax2.axis("scaled")
ax2.set_xlim(0, img_size[0])
ax2.set_ylim(0, img_size[1])
fig.colorbar(pos, ax=ax2)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

i=np.random.randint(len(imgpoints))
print(i)
fig, ((ax, ax_im)) = plt.subplots(1,2)
color = 'tab:blue'
x = imgpoints[i][:,0,0]
y = imgpoints[i][:,0,1]
# scale = 200.0 * np.random.rand(n)
ax.scatter(x, y, c=color, alpha=0.5, label="original", edgecolors='none')

color = 'tab:orange',
imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
x = imgpoints2[:,0,0]
y = imgpoints2[:,0,1]
ax.scatter(x, y, c=color, alpha=0.5, label="reprojection", edgecolors='none')

ax.legend()
ax.grid(True)

im = cv2.imread(idx2image_path[i])
ax_im.imshow(im)

plt.show()


In [ ]:
imgpoints2[:,0,1]

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)
    total_error += error*error
print( "rms error: {}".format(np.sqrt(total_error/(len(objpoints)*len(imgpoints2)))))

tot_error=0
total_points=0
for i in range(len(objpoints)):
    reprojected_points, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    reprojected_points=reprojected_points.reshape(-1,2)
    tot_error+=np.sum(np.abs(imgpoints[i]-reprojected_points)**2)
    total_points+=len(objpoints[i])

mean_error=np.sqrt(tot_error/total_points)
print ("Mean reprojection error: ", mean_error)




In [ ]:
%matplotlib inline

# Test undistortion on an image
img = cv2.imread(random.choice(image_paths))
img_size = (img.shape[1], img.shape[0])
print(img.shape[:2])
# Do camera calibration given object points and image points


DIM=img_size
h,w = img.shape[:2]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(mtx, dist, np.eye(3), mtx, DIM, cv2.CV_16SC2)
undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)


ret, corners = cv2.findChessboardCorners(img, (CHECKERBOARD[0],CHECKERBOARD[1]), None)

# If found, add object points, image points
print("RET",ret)
if ret == True:
    subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
    cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
    cv2.drawChessboardCorners(img, (CHECKERBOARD[0],CHECKERBOARD[1]), corners, ret)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)

#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(30,15))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(undistorted_img)
ax2.set_title('Undistorted Image', fontsize=30)

In [ ]:
CHECKERBOARD = (10,7)
img_size = (1280, 960)
# img_size = (640, 480)
# images_glob = '/catkin_ws/src/dataset/calib/fish640x480_1/*.jpeg'
images_glob = '/catkin_ws/src/dataset/calib/fish1280x960A3/**/*.png'

FISHEYE = 'fisheye'
PERSPECTIVE = 'perpective'
MODEL = FISHEYE
H_FLIP = False

def flip(img):
    if H_FLIP:
        img = cv2.flip(img, 0)
    return img

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1],3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1,2)

objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob(images_glob)
print('found {} images'.format(len(images)))

used_image_count = 0
# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = flip(cv2.imread(fname))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (CHECKERBOARD[0],CHECKERBOARD[1]), None)

    # If found, add object points, image points
    if ret == True:
        used_image_count += 1
        objpoints.append(objp)
        imgpoints.append(corners)

        # # Draw and display the corners
        # cv2.drawChessboardCorners(img, (CHECKERBOARD[0],board_height), corners, ret)
        # #write_name = 'corners_found'+str(idx)+'.jpg'
        # #cv2.imwrite(write_name, img)
        # cv2.imshow('img', img)
        # cv2.waitKey(500)

print("Found corners on {} images".format(used_image_count))
cv2.destroyAllWindows()

In [ ]:
import pickle
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
ret, K, D, rvecs, tvecs = cv2.fisheye.calibrate(objpoints, imgpoints, img_size,None,None, flags=calibration_flags)
print("RET", ret)
D_pickle = {}
D_pickle["K"] = K
D_pickle["D"] = D
pickle.dump( D_pickle, open( "camera_calibration_pickle.p", "wb" ) )

print(K, '\n', D)
print("Camera.fx:", K[0][0])
print("Camera.fy:", K[1][1])
print("Camera.cx:", K[0][2])
print("Camera.cy:", K[1][2])

print('\n')
print('Camera.k1:', D[0][0])
print('Camera.k2:', D[1][0])
print('Camera.k3:', D[2][0])
print('Camera.k4:', D[3][0])


In [ ]:


Camera.fx: 736.799380041753
Camera.fy: 738.1296110449435
Camera.cx: 623.0269851507727
Camera.cy: 479.41796594759063


Camera.k1: -0.011885462079235002
Camera.k2: -0.04402188574632658
Camera.k3: 0.06925286809391396
Camera.k4: -0.0457320089421333

In [ ]:
import cv2
# assert cv2.__version__[0] == '3', 'The fisheye module requires opencv version >= 3.0.0'
import numpy as np
import os
import glob
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob(images_glob)
used_image_count = 0
for fname in images:
    img = flip(cv2.imread(fname))
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD)
    # print("{} - found {} corners".format(fname, len(corners) if ret == True else 0))
    # If found, add object points, image points (after refining them)
    if ret == True:
        used_image_count += 1
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
print("Found corners on {} images".format(used_image_count))


In [ ]:

N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
# rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
# tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
ret, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        # rvecs,
        # tvecs,
        flags=calibration_flags,
        # (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("RET", ret)
print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print(K, '\n', D)
print("Camera.fx:", K[0][0])
print("Camera.fy:", K[1][1])
print("Camera.cx:", K[0][2])
print("Camera.cy:", K[1][2])

print('\n')
print('Camera.k1:', D[0][0])
print('Camera.k2:', D[1][0])
print('Camera.k3:', D[2][0])
print('Camera.k4:', D[3][0])


In [ ]:

Camera.fx: 736.5015466535494
Camera.fy: 737.7523633241663
Camera.cx: 622.7962516403339
Camera.cy: 479.59388922989496


Camera.k1: -0.010955659973999754
Camera.k2: -0.047002618776998864
Camera.k3: 0.059114596046676536
Camera.k4: -0.032725046003063904


Camera.fx: 736.799380041753
Camera.fy: 738.1296110449435
Camera.cx: 623.0269851507727
Camera.cy: 479.41796594759063


Camera.k1: -0.011885462079235002
Camera.k2: -0.04402188574632658
Camera.k3: 0.06925286809391396
Camera.k4: -0.0457320089421333

In [ ]:
print(K)
print("fx:", K[0][0])
print("fy:", K[1][1])
print("cx:", K[0][2])
print("cy:", K[1][2])

print('\n\n', D)
print('k1:', D[0][0])
print('k2:', D[1][0])
print('k3:', D[2][0])
print('k4:', D[3][0])